### Search in Rotated Sorted Array

Challenge is: binary search algo on a sorted array is O(logN), but the same "time complexity" is required for a sorted array rotated at some index. 

e.g. [1,2,3,4,5,6,7,8] rotated at 7(index 6) -> [7,8,1,2,3,4,5,6]


LeetCode: https://leetcode.com/problems/search-in-rotated-sorted-array/

#### First attempt

Algorithm:
The main idea is to try to locate the pivot point - then turn it into a simple binary search by either looking at the left of right of the pivot point (both subarray are sorted)

- compare first elem with last elem
    - if first < last: the array is not rotated, do normal binary search - O(logN)
    - otherwise: 
        * do binary search looking for the pivot index - O(logN)
        * the rotated array can then be broken into two sorted arrays - equivalent to the end of the first iteration of a normal binary search on a sorted array
        * compare target with first elem to determin which 'sorted half' the normal binary search should perform
        * perform binary search on the 'sorted half' - O(logN)

The combined time complexity(worst): O(logN + logN) = O(logN)

In [2]:
def search(nums, target):
    """
    :type nums: List[int]
    :type target: int
    :rtype: int
    """
    if not nums:
        return -1
    elif len(nums) == 1:
        if nums[0] == target:
            return 0
        else:
            return -1

    if nums[0] > nums [-1]:
        # Rotated

        # Search pivot point:  O(logN)
        left = 0 
        right = len(nums) - 1

        pivot = None

        while left < right:
            mid = (left + right) // 2
            #print(left, mid, right)

            if nums[mid-1] > nums[mid]:
                #print('Found pivot', mid, nums[mid-1], nums[mid])
                # found pivot
                pivot = mid
                break

            if nums[mid] < nums[-1]:
                # search left
                right = mid - 1
            else:
                left = mid + 1  
        else:
            # pivot is the index of the left most elem before rotation
            pivot = left
        
        # Set high/low for the target search
        if target < nums[0]:
            low = pivot
            high = len(nums) - 1
        else:
            low = 0
            high = pivot - 1

    else:
        # Not Rotated - set the pivot so the search code works.
        low = 0
        high = len(nums) - 1

    #print('Pivot', pivot)

    # Search if either left or right of the pivot point
    while low < high:
        mid = (low + high) // 2

        if nums[mid] == target:
            return mid
        elif nums[mid] > target:
            high = mid - 1
        else:
            low = mid + 1

    if nums[low] == target:
        return low

    return -1

In [3]:
%timeit search([4,5,6,7,8,1,2,3], 8)

11.7 µs ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Second attempt

Algorithm:
The main idea is to iterate like a simple binary search. Except that more conditions are checked to determine which half of the array to look up next.

e.g. when the target < mid, it is on the left subarray if targer > first, otherwise on the right subarray

Time complexity(worst): O(logN)

In [26]:
def search(nums, target):
    """
    :type nums: List[int]
    :type target: int
    :rtype: int
    """
    if not nums:
        return -1
    
    left = 0
    right = len(nums) - 1
    
    while left < right:
        mid = (left + right) // 2
        
        if nums[mid] == target:
            return mid
        elif nums[mid] < target:
            # when mid < target - we need to check if mid is < first or not 
            # if it is > first, that means mid is on the right of pivot before rotation
            if nums[0] <= nums[mid]:
                left = mid + 1
            else:
                # otherwise, needs compare target with last
                if target == nums[-1]:
                    return len(nums) - 1
                elif target < nums[-1]:
                    left = mid + 1
                else:
                    right = mid - 1
        else:  # nums[mid] > target
            if nums[0] <= nums[mid]:
                if target == nums[0]:
                    return 0
                elif target < nums[0]:
                    left = mid + 1
                else:
                    right = mid - 1
            else:
                right = mid - 1
                

    if nums[left] == target:
        return left
    else:
        return -1
    
search([3,4,5,1,2], 4)

1

In [27]:
%timeit search([4,5,6,7,8,1,2,3], 8)

8.39 µs ± 8.11 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
